# 0) Instructions:
Please complete the workbook below. Some of the calculations are already ready to be "run". However, please read the text carefully to find questions that you should answer for credit. Remember that to answer a question in text, you click "insert", then "insert cell below", switch the input from "code" to "markdown", type your answer, and finally click the run button to set your text in stone. In a few questions, you will need to do some calculations on your own. Hint: for these calculations you can copy, paste, and modify code that is above the calculation that you need to do. You may work by yourself or in groups of 2. Please remember to put your name on top, remember to save the workbook, and remember to upload to Canvas.

**Please read** Please run the code below to install the "DescTools" package. **Do not proceed until the star goes away! Do not click run twice**!

In [ ]:
install.packages('DescTools', dep = TRUE)
print('Done with DescTools install')

Now we will load one package "DescTools" that has some functions we will need for making comparisons following our ANOVA, and the "tidyverse" which actually loads a few usefull packages. 

In [ ]:
library(tidyverse)
library(DescTools)
options(repr.plot.width = 5, repr.plot.height = 4)

# 1) Back to the chickens
In the last lab, we conducted an Analysis of Variance for determining if there was evidence that *at least* one of the population means of chicken weights differed by feed type. Now the all important question is which of the means are different from the others, or more specifically we will want to test which pairs of means are different from one another. Before we get to making these comparisons, let's reload the data, make a boxplot to reorient ourselves, and conduct the ANOVA again!

First we will load the "chickwts" dataset and take a sample of size 10 from each group again:

In [ ]:
data(chickwts)
chickwts <- chickwts %>% group_by(feed) %>% slice(1:10)

Now to make a boxplot showing the distribution of chicken weights by feed type. Remeber that the means are shown as red triangles: 

In [ ]:
ggplot(chickwts, aes(x = feed, fill = feed, y = weight)) + geom_boxplot(alpha = .5) + 
  stat_summary(fun.y = mean, colour = "darkred", geom = "point", shape = 18, size = 3, show.legend = FALSE) +
  theme_bw()

**Question for you** For which of the pairs of groups do you think the difference in the means will be greatest? For which of the pairs of groups do you think the differences in the means will be smallest? Please answer these questions in a new markdown cell below. 

Returning to the ANOVA, the hypotheses being tested were $H_0: \mu_1 = \mu_2 = ... = \mu_p$ versus $H_a:$ At least one $\mu_j$ is not equal to the others. 

**Question for you.** Describe in words what the test statistic for the ANOVA overall $F$-test is. Please answer below in a new markdown cell. 

**Question for you.** What would the rejection rule be for this ANOVA $F$-test? Please actually find the critical value from the $F$ distribution. You can use R to do this or the textbook. Please answer below in a new markdown cell. 

Okay, let's generate the ANOVA again using R. We will save this object that contains our results as "anova1":

In [ ]:
anova1 <- aov(weight ~ feed, data = chickwts)

Now to get our ANOVA table we can use the "summary" function, which will return the sum of squares, the mean squares, the $F^*$ test statistic, and the p-value from the $F$ test:

In [ ]:
summary(anova1)

**Question for you**. In the above output you find a p-value, labeled as "Pr(>F)". What does this p-value represent? What does it tell you about our original research question as to whether the choice of feed impacts chicken weights? 

## 1.1) Pairwise comparisons
Okay, since we have observed significant evidence that at least one of the population means is different from the others, let's now compare each of the means (pairwise comparisons). We can start by taking a look at each of the sample means within each group: 

In [ ]:
aggregate(weight ~ feed, data = chickwts, mean)

Let's save this set of means by group as "groupMeans" as we will need them later to make pairwise comparisons:

In [ ]:
groupMeans <- aggregate(weight ~ feed, data = chickwts, mean)

### 1.1.1) Fisher's Least Significant Difference
The first method for making pairwise comparisons that we will discus is the Fisher's Least Significant Difference (LSD) approach. This approach does not preserve the experimentwise type I error probability below a specified $\alpha$. In other words, if you set $\alpha = 0.05$, for the whole set of pairwise comparisons that you make, the experimentwise type I error proabability will be greater than $\alpha$. However, this approach is superior to doing pairwise t-tests as we have learned in earlier lectures when we were comparing the means of two populations; This is due to the fact that the within group mean square, aka $s_W^2$, is a better estimate of the variance (assuming equal variances) than the square of the pooled standard deviation estimate that we learned earlier, $s_p$.

**Question for you.** In practice, what does "the experimentwise type I error proabability will be greater than $\alpha$" mean? Please explain what it means for both hypothesis tests and confidence intervals. Please answer this question below in a new markdown cell with one or a few sentances. 

Let's start off by comparing the means between chicken weights for the chickens fed horsebean versus casein. We will determine a point estimate and confidence interval for $\mu_{\text{horsebean}} - \mu_{\text{casein}}$. For the point estimate we simply contrast the sample means. 

In [ ]:
160.2 - 326.8

Now to estimate a 95% confidence interval for $\mu_{\text{horsebean}} - \mu_{\text{casein}}$, we will use $\bar{y}_1 - \bar{y}_2 \pm t_{\alpha / 2, N-k} \sqrt{\frac{SSW}{N-k}\left(\frac{1}{n_1} + \frac{1}{n_2} \right)}$:

In [ ]:
(160.2 - 326.8) - qt(0.025, df = 60 - 6, lower.tail = FALSE) * sqrt(3109 * (1/10 + 1/10))

In [ ]:
(160.2 - 326.8) + qt(0.025, df = 60 - 6, lower.tail = FALSE) * sqrt(3109 * (1/10 + 1/10))

**Question for you.** Repeating the process we have completed abouve for determining a point estimate and 95% confidence interval for $\mu_{\text{horsebean}} - \mu_{\text{casein}}$, determine a point estimate and confidence interval for $\mu_{\text{soybean}} - \mu_{\text{linseed}}$. You may copy, paste, and modify the code from above into new cells to do this. At the end please add a new markdown cell to report what the point estimate and 95% confidence interval are. 

Up to this point, we have made two pairwise comparisons of population means. Later we will use software to make all 15 pairwise comparisons. But for now, let's discuss statistical hypothesis tests. To make pairwise hypothesis tests comparing the means of two of the populations we can use the LSD method. Remeber that this method will not preserve the experimental Type I error rate at the specified $\alpha$. 

Let's start by testing $H_0: \mu_{\text{horsebean}} - \mu_{\text{casein}}= 0$ versus $H_a:\mu_{\text{horsebean}} - \mu_{\text{casein}} \neq 0$. To do this we will need to specify an $\alpha$ (let's use 0.05), a test statistic, and a rejection rule. The test statistic is: $t^*=\frac{\bar{y}_1 - \bar{y}_2}{\frac{SSW}{N-k}\left(\frac{1}{n_1} + \frac{1}{n_2} \right) }$.

**Question for you**. Please state the rejection rule for this test below in a new markdown cell. Please actually find the crictical value (you may use R to do this of course!).

Okay, so now to calculate our test statistic:

In [ ]:
(160.2 - 326.8) / sqrt(3109 * (1/10 + 1/10))

**Question for you**. What is the conclusion of this invidual pairwise comparison? Please answer below in a new markdown cell.

Now to determine the level of significance, or p-value, we will determine $2\times P(t \geq |t^*|)$ using a $t$ distribution with $N-k$ degrees of freedom. First let's find $\times P(t \geq |t^*|)$.

In [ ]:
pt((160.2 - 326.8) / sqrt(3109 * (1/10 + 1/10)), df = 60 - 6, lower.tail = TRUE)

And now let's find $2 \times P(t \geq |t^*|)$.

In [ ]:
2 * pt((160.2 - 326.8) / sqrt(3109 * (1/10 + 1/10)), df = 60 - 6, lower.tail = TRUE)

**Question for you**. Using the Fisher's LSD method, conduct a statistical hypothesis test to determine if there is evidence that the population mean weights are different for chickens fed linsead versus horsebean. You may copy and paste the codes from above into new code cells. After you conduct the test, please state your conclusion in a new markdown cell. 

Now to do make all of the pairwise comparisons at once, we can use the "PostHocTest" function from the "DescTools" package (yes the one that takes forever to install). This function takes the ANOVA object that we created earlier as the first argument, and then requires you to specify the method for making pairwise comparisons. In our case, we will use 'method = "lsd"'. 

In [ ]:
PostHocTest(anova1, method = "lsd")

## 1.1.2) The Bonferroni procedure
The problem with the Fisher's LSD procedure is that it does not preserve the experimentwise Type I error probability at the specified $\alpha$. The most straightforward (and naive) way to ensure that the Type I error probability is actually maintained at $\alpha$ over the whole set of pairwise comparisons made is to simply divide $\alpha$ by the number of pairwise comparisons. We will call this new value $\alpha_i$ meaning if there are $m$ pairwise comparisons we set $\alpha_i = \alpha_E / m$. 

For our chicken weights example, there are 6 different types of feeds. So to make all of the possible pairwise comparisons of mean chicken weights by group we will have 15 tests. 15 is the number of ways that you can choose a set of two objects from a set of 6. As the total number of populations evaluated in the ANOVA increases, it can be hard to count all of the pairwise comparisons. We can use the "choose" function in R to tell us how many there are. For example to count the number of ways to choose a set of two objects out of 6 we do:

In [ ]:
choose(6, 2)

Now, before we determine confidence intervals for the difference in population means and conduct hypothesis tests to determine for which comparisons we have evidence of a difference in means we need to find $\alpha_i = \alpha_E / m$:

In [ ]:
0.05 / 15

To make our lives easier, let's save this value as "alphaI":

In [ ]:
alphaI <- 0.05 / 15

Okay, now let's estimate a new 95% confidence ineterval for $\mu_{\text{horsebean}} - \mu_{\text{casein}}$ using the Bonferroni procedure. We will use the same formula as before with the Fisher's LSD method, however, we will need to change the value of $\alpha$ when we are finding the quantile of the $t$ distribution that we need: 

In [ ]:
(160.2 - 326.8) - qt(alphaI / 2, df = 60 - 6, lower.tail = FALSE) * sqrt(3109 * (1/10 + 1/10))

In [ ]:
(160.2 - 326.8) + qt(alphaI / 2, df = 60 - 6, lower.tail = FALSE) * sqrt(3109 * (1/10 + 1/10))

Now let's conduct the hypothesis test again of $H_0: \mu_{\text{horsebean}} - \mu_{\text{casein}}=0$ versus $H_a: \mu_{\text{horsebean}} - \mu_{\text{casein}}\neq 0$. We will have the *same* test statistic as before, however we will need a new critical value. 

**Question for you.** What is this new critical value considering our new $\alpha_i$? Please answer below in a markdown cell. You can use software to find this critical value. 

When we are conducting statistical hypothesis tests, we have two equivalent ways to determine our conclusion. The first is to see if the test statistic lies in the rejection region; the second is to directly compare the p-value for the test to the specified $\alpha$ value. When we use the Bonferroni procedure, we also have two equivalent ways to determine our conclusion. The first is to determine the rejection region that would correspond to our lower $\alpha_i$ value. The second would be to compare $m \times \text{p-value}$ with $\alpha_E$. For this way, we compute the p-value as we would for the Fisher's LSD method, but we then multiply it by the total number of comparisons we will make and compare this adjusted p-value to the overall experimentwise Type I error probability, $\alpha_E$. We call this new p-value a Bonferroni corrected or Bonferroni adjusted p-value.

To do this for our test of $H_0: \mu_{\text{horsebean}} - \mu_{\text{casein}}=0$, we can take the p-value we computed before and multiply it by the number of comparisons we are going to make (15): 

In [ ]:
15 * 2 * pt((160.2 - 326.8) / sqrt(3109 * (1/10 + 1/10)), df = 60 - 6, lower.tail = TRUE)

**Question for you**. Determine the Bonferroni corrected p-value for the test of $H_0: \mu_{\text{soybean}} - \mu_{\text{linseed}} =0 $ versus $H_a: \mu_{\text{soybean}} - \mu_{\text{linseed}}\neq 0$, considering that we will make all possible pairwise comparisons. Please insert a new code cell and determine this corrected p-value.

Okay, now to use software to determine Bonferroni corrected p-values and confidence intervals we can use the same "PostHocTest" function as before, however we will change the method from "lsd" to "bonf" for Bonferroni: 

In [ ]:
PostHocTest(anova1, method = "bonf")

## 1.1.3) The Tukey Honestly Significant Difference (HSD) procedure
The Bonferroni procedure is known to be "too conservative". This means that if you specify an experimentwise Type I error probability $\alpha_E$, and then you use the Bonferonni procedure, the actual $\alpha_E$ that you acheive is lower than what you specify. So it solves the problem of the actual $\alpha_E$ being greater than the specified $\alpha_E$, by creating the opposite problem.

**Question for you** Why would "the actual $\alpha_E$ that you acheive being lower than the $\alpha_E$ you specify" be a problem? Please answer this question in a new markdown cell below. 

The Tukey HSD procedure is somewhat of a comprimise choice for when you want to make pairwise comparisons. If the assumptions we make for the overall ANOVA $F$-test are met (normally distributed populations, homogeneity of variances, and independence of each sample), then the Tukey HSD procedure can maintain the experimentwise $\alpha_E$ at or below a specified $\alpha_E$, *and*, the procedure will be less conservative than the Bonferroni approach. We don't discuss how to use the HSD procedure by hand, however it is a good procedure (often the best for one-way ANOVA) for you to be able to employ. To make Tukey HSD confidence intervals and adjusted p-values, we can use the "TukeyHSD" function. This function will take our ANOVA object that we created before as it's argument: 

In [ ]:
TukeyHSD(anova1)

**Question for you**. In comparing the confidence intervals from the three methods we have discussed, Fisher's LSD, Bonferroni, and Tukey HSD, which are systematically the most narrow, and which are systematically the most wide. Please answer this question in a new markdown cell below. 

**Question for you**. If you wanted to preserve the experimentwise Type I error probability at 0.05 in comparing the chicken weights by feed type, what procedure would you use and why? 

# 2) Comparing urinary arsenic between different racial groups

Arsenic is a heavy metal that is toxic to humans. We are exposed to different heavy metals (including arsenic) in the food that we eat and the water that we drink. Some researchers are interested in whether arsenic exposure differs between racial groups, as there may be different dietary patterns within these groups. For examle, some racial groups may *on average* or typically consume more rice than others (a possible route of having small exposures to arsenic) or more seafood (another possible route of exposure depending on the type of fish and how it was raised). For this next example, let's assume that we are interested in determining whether there is evidence that the population means of urinary arsenic levels differ by ethnicity.

I took a very small subset of the data from the National Health and Nutrition Examination Survey (https://www.cdc.gov/nchs/nhanes/index.htm). This study is "a program of studies designed to assess the health and nutritional status of adults and children in the United States" that has been running since the 1960's. Some of the de-identified datasets are publically availble. For our example we will look at a subset that I made for studying the relationship between urinary arsenic levels and ethnicity. **Caution** NHANES is designed to be used for making national level estimates and hypothesis tests that are valid at the nationwide level. However, to make this nationally representative you have to use a special system of weighting each observation differently. We will not do this for this example *hence* the inferences we make will not generalize to the population of the US. This subset is from the 2015-2016 cycle of this study. The data contains measurements of arsenic that were made from the participant's urine as well as demographic information about the included participants.

First, we can retrieve the data from my NMSU site. Please run the following block of code below to retrieve and load the dataset as a "data.frame":

In [ ]:
dataURL <- "https://wordpress.nmsu.edu/ptrainor/files/2019/10/NHANES_Sample.xlsx"
httr::GET(dataURL, httr::write_disk(tf <- tempfile(fileext = ".xlsx")))
df1 <- as.data.frame(readxl::read_excel(tf, na = 'NA'))
if(nrow(df1) > 0) print("The NHANES data is loaded")

## 2.1) Some exploratory analyses
The NHANES study has weird variable names. For us, we are looking for a variable called "urxuas", which is "urinary total arsenic" measured in µg/L. Let's first make a histogram showing the distribution of the "urinary total arsenic":

In [ ]:
ggplot(df1, aes(x = urxuas)) + 
   geom_histogram(bins = 50, color = "black", fill = "grey80") + 
   theme_bw()

Now, let's make a qqplot of the same data:

In [ ]:
qqnorm(df1$urxuas)
qqline(df1$urxuas)

**Question for you** Given the histogram and Q-Q plot that you have just made of the total urinary arsenic in these human subjects, how would you describe the distribution of the data? Please comment on it's relationship to a normal distribution, if there is skew present (and if so in which direction).

## 2.2) A variable transformation
We will discuss this further later in the course, but occasionally when we have data that does not appear to follow a normal distribution, we can utilize a variable transformation to see if the transformed data would be closer in distribution to a normal distribution. In other words, rather than conduct the ANOVA using the original measurements, we might try to use the transformed measurements after taking the natural log of the measurements or after taking the square root. For this example, let's try taking the natural log (in R this is just "log"). We will create a new variable and add it to the "data.frame" called "LogUrinaryAs" that is the log-transformed values of the original data: 

In [ ]:
df1$LogUrinaryAs <- log(df1$urxuas)

**Question for you.** Using the codes from above for making a histogram and Q-Q plot (and copying, pasting, and modifying them), please make a histogram and Q-Q plot for the log-transformed values.

**Question for you.** Do the transformed values appear to be closer in distribution to a normal distribution than the values on the original measurement scale?

Okay, now recall that our research question is to see if the levels of urinary arsenic differ between ethnicities. We will want to do an ANOVA $F$-test using the log-transformed data and we will want to make pairwise comparisons between the ethnicities. 

**Caution**, when we conduct these statistical tests, they will be for determining differences in the population means on the log-scale data, rather than on the original scale. This is often more appropriate when we have data that appears closer to being approximately normally distributed on the log-scale.

The variable in the NHANES data that records the ethnicity of each subject is called "ridreth3". Let's take a look at what this variable looks like. We can use the "head" function to print out the first few observations of this variable. Remember that we use the "\\$" notation to retrieve a variable from a "data.frame". For example, "df1$ridreth3" will retrieve the "ridreth3" variable from the "data.frame" named "df1" for us:

In [ ]:
head(df1$ridreth3)

Okay, so you will notice that the variable that tells us the ethnicity of each of the study subjects is represented as integers. This is very common in large datasets. Ethnicity is a nominal variable that has levels such as "Non-Hispanic Asian". Each integer in the data needs to be mapped to the appropriate level. We can find the documentation on this mapping (and other demographics) here: https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm. For us, we will make a new variable called "Ethnicity" that is a copy of the "ridreth3" variable and then we will map these numbers to the different levels. Fist we will create this new variable as a copy of the other:

In [ ]:
df1$Ethnicity <- df1$ridreth3

Now, we will convert this variable from a list of integer to a "factor" variable. This is R's way of saying a nominal variable. The "as.factor" variable will convert this variable from a set of integers to a nominal variable with different levels:

In [ ]:
df1$Ethnicity <- as.factor(df1$Ethnicity)

Let's take a look at what these different levels are. The "levels" function will return for us what the different levels are that can be found in the nominal variable "Ethnicity":

In [ ]:
levels(df1$Ethnicity)

We now need to map these levels to the levels that we find in the NHANES documentation. We will do this by telling R that "1" should be replaced by "Mexican American", "2" should be replaced by "Other Hispanic" and so on:

In [ ]:
levels(df1$Ethnicity) <- c("Mexican American", "Other Hispanic", "Non-Hispanic White",
                          "Non-Hispanic Black", "Non-Hispanic Asian", 
                           "Other Race - Including Multi-Racial")

Okay, now let's look at the first 6 observations of this "Ethnicity" variable:

In [ ]:
head(df1$Ethnicity)

Now for our own interest, let's count how many subjects of each ethnicity we have. We can do this using the table function:

In [ ]:
table(df1$Ethnicity)

Okay, now let's make a boxplot of the log-transformed urinary arsenic data colored by each ethnicity:

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)
ggplot(df1, aes(x = Ethnicity, y = LogUrinaryAs, fill = Ethnicity)) + geom_boxplot() + 
   theme_bw() + theme(axis.text.x=element_blank())

Let's also determine the sample means of the log-transformed urinary total arsenic by group:

In [ ]:
aggregate(LogUrinaryAs ~ Ethnicity, data = df1, mean)

Now let's create the ANVOA object so that we can see the ANOVA table and determine the result of our ANOVA overall $F$-test:

In [ ]:
anova2 <- aov(LogUrinaryAs ~ Ethnicity, data = df1)

The "summary" function will provide a summary table of the ANOVA for us. 

In [ ]:
summary(anova2)

Now, let's use Tukey's method to determine confidence intervals for the difference in population means:

In [ ]:
TukeyHSD(anova2)

**Question for you.** Which ethnicities appear to have different levels of urinary total arsenic from each other? Please answer in a new markdown cell below. 